# RT Auto-Contouring: Data Exploration

This notebook provides tools for exploring radiotherapy contouring datasets.

## Contents
1. Setup and Imports
2. Load Configuration
3. Explore DICOM Data
4. Visualize Structures
5. Quality Assessment Summary
6. Dataset Statistics

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import SimpleITK as sitk
import pydicom
from IPython.display import display

# Project imports
from config.paths import get_path_config
from config.structures import (
    get_all_canonical_names,
    get_volume_range,
    STRUCTURE_COLORS,
    STRUCTURE_LABELS
)
from src.data_quality.quality_assessment import DatasetQualityAssessment
from src.preprocessing.dicom_to_nifti import DicomToNiftiConverter, find_dicom_files

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("Imports successful!")

## 2. Load Configuration

In [ ]:
# Get path configuration
path_config = get_path_config()

print("Path Configuration:")
print(f"  Project root: {path_config.project_root}")
print(f"  Data root: {path_config.data_root}")
print(f"  DICOM raw: {path_config.dicom_raw}")
print(f"  nnUNet root: {path_config.nnunet_root}")

# Display structure configuration
print("\nTarget Structures:")
for name in get_all_canonical_names():
    label = STRUCTURE_LABELS[name]
    color = STRUCTURE_COLORS[name]
    vol_range = get_volume_range(name)
    print(f"  {label}. {name}: RGB{color}, volume range: {vol_range[2]:.1f}-{vol_range[3]:.1f} cc")

## 3. Explore DICOM Data

Set your DICOM data directory path:

In [ ]:
# Set DICOM directory (change this to your data location)
dicom_root = path_config.dicom_raw

# Find patient directories
if dicom_root.exists():
    patient_dirs = sorted([d for d in dicom_root.iterdir() if d.is_dir()])
    print(f"Found {len(patient_dirs)} patient directories")
    
    # Display first few
    if patient_dirs:
        print("\nFirst 5 cases:")
        for i, d in enumerate(patient_dirs[:5]):
            print(f"  {i+1}. {d.name}")
else:
    print(f"DICOM directory does not exist: {dicom_root}")
    print("Please update the dicom_root path above.")

### Examine a Single Case

In [ ]:
# Select a case to examine (change index as needed)
if patient_dirs:
    case_dir = patient_dirs[0]
    print(f"Examining case: {case_dir.name}")
    
    # Find CT and RTSTRUCT
    ct_series_path, rtstruct_path = find_dicom_files(case_dir)
    
    if ct_series_path:
        print(f"\nCT series found: {ct_series_path}")
        
        # Load CT series
        reader = sitk.ImageSeriesReader()
        dicom_names = reader.GetGDCMSeriesFileNames(str(ct_series_path))
        print(f"  Number of slices: {len(dicom_names)}")
        
        # Read first slice for metadata
        dcm = pydicom.dcmread(dicom_names[0], stop_before_pixels=True)
        print(f"  Patient ID: {dcm.PatientID if hasattr(dcm, 'PatientID') else 'N/A'}")
        print(f"  Study Date: {dcm.StudyDate if hasattr(dcm, 'StudyDate') else 'N/A'}")
        print(f"  Slice Thickness: {dcm.SliceThickness if hasattr(dcm, 'SliceThickness') else 'N/A'} mm")
    
    if rtstruct_path:
        print(f"\nRTSTRUCT found: {rtstruct_path}")
        
        # Load RTSTRUCT
        from rt_utils import RTStructBuilder
        rtstruct = RTStructBuilder.create_from(
            dicom_series_path=str(ct_series_path),
            rt_struct_path=str(rtstruct_path)
        )
        
        roi_names = rtstruct.get_roi_names()
        print(f"  Number of ROIs: {len(roi_names)}")
        print(f"  ROI names: {roi_names}")

## 4. Visualize Structures

Visualize CT and structure contours:

In [ ]:
def visualize_slice(ct_image, masks_dict, slice_idx=None, title="CT with Structures"):
    """
    Visualize CT slice with structure overlays.
    
    Args:
        ct_image: SimpleITK Image
        masks_dict: Dictionary mapping structure names to binary masks
        slice_idx: Slice index to visualize (None for middle slice)
        title: Plot title
    """
    ct_array = sitk.GetArrayFromImage(ct_image)
    
    if slice_idx is None:
        slice_idx = ct_array.shape[0] // 2
    
    # Get CT slice
    ct_slice = ct_array[slice_idx, :, :]
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Display CT
    ax.imshow(ct_slice, cmap='gray', vmin=-200, vmax=200)
    
    # Overlay structures
    for struct_name, mask in masks_dict.items():
        mask_slice = mask[slice_idx, :, :]
        
        if np.any(mask_slice):
            # Get color
            color = STRUCTURE_COLORS.get(struct_name, (255, 0, 0))
            color_normalized = tuple(c/255.0 for c in color)
            
            # Create contour overlay
            contours = plt.contour(
                mask_slice,
                levels=[0.5],
                colors=[color_normalized],
                linewidths=2
            )
            
            # Add label
            contours.collections[0].set_label(struct_name)
    
    ax.set_title(f"{title} - Slice {slice_idx}")
    ax.axis('off')
    ax.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
    
    plt.tight_layout()
    plt.show()

# Example usage (if case data is loaded)
if ct_series_path and rtstruct_path:
    # Load CT
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(str(ct_series_path))
    reader.SetFileNames(dicom_names)
    ct_image = reader.Execute()
    
    # Extract masks
    masks = {}
    for roi_name in roi_names:
        try:
            mask = rtstruct.get_roi_mask_by_name(roi_name)
            masks[roi_name] = np.array(mask)
        except:
            pass
    
    # Visualize
    visualize_slice(ct_image, masks, title=f"Case: {case_dir.name}")

## 5. Quality Assessment Summary

Run quality assessment on the dataset:

In [ ]:
# Run quality assessment (this may take a while)
if dicom_root.exists() and patient_dirs:
    print("Running quality assessment...")
    print("This may take several minutes depending on dataset size.")
    
    qa = DatasetQualityAssessment(
        dicom_root=dicom_root,
        output_dir=path_config.quality_reports
    )
    
    # Process a subset for quick exploration (remove limit for full dataset)
    for patient_dir in patient_dirs[:10]:  # Process first 10 cases
        case_id = patient_dir.name
        
        try:
            ct_series_path, rtstruct_path = find_dicom_files(patient_dir)
            
            if ct_series_path and rtstruct_path:
                qa.assess_case(
                    case_id=case_id,
                    ct_series_path=ct_series_path,
                    rtstruct_path=rtstruct_path
                )
        except Exception as e:
            print(f"Error processing {case_id}: {e}")
    
    # Generate report
    df = qa.generate_report()
    
    print("\nQuality Assessment Complete!")
    display(df.head())

### View Quality Metrics

In [ ]:
if 'df' in locals() and not df.empty:
    # Quality score distribution
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    axes[0].hist(df['quality_score'], bins=20, edgecolor='black')
    axes[0].set_xlabel('Quality Score')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Quality Score Distribution')
    axes[0].grid(True, alpha=0.3)
    
    # Number of issues
    axes[1].hist(df['num_issues'], bins=20, edgecolor='black')
    axes[1].set_xlabel('Number of Issues')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Issues per Case')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\nQuality Metrics Summary:")
    print(df[['quality_score', 'num_issues']].describe())

## 6. Dataset Statistics

Analyze structure volumes and presence:

In [ ]:
if 'df' in locals() and not df.empty:
    # Structure presence
    presence_cols = [col for col in df.columns if col.endswith('_present')]
    
    if presence_cols:
        presence_data = df[presence_cols].sum()
        presence_data.index = [col.replace('_present', '') for col in presence_data.index]
        
        fig, ax = plt.subplots(figsize=(12, 6))
        presence_data.plot(kind='bar', ax=ax)
        ax.set_xlabel('Structure')
        ax.set_ylabel('Number of Cases')
        ax.set_title('Structure Presence Across Dataset')
        ax.grid(True, alpha=0.3, axis='y')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    
    # Volume distributions
    volume_cols = [col for col in df.columns if col.endswith('_volume_cc')]
    
    if volume_cols:
        print("\nVolume Statistics (cc):")
        volume_stats = df[volume_cols].describe()
        volume_stats.columns = [col.replace('_volume_cc', '') for col in volume_stats.columns]
        display(volume_stats)

## Next Steps

After exploring the data:

1. Run full quality assessment using `scripts/run_quality_assessment.py`
2. Prepare nnU-Net dataset using `scripts/run_preprocessing.py`
3. Train models using `scripts/run_training.py`
4. Run inference using `scripts/run_inference.py`